# 잔차의 해석

In [1]:
import pandas 
from statsmodels.formula.api import ols

In [3]:
child = pandas.read_csv('child.csv')

In [4]:
child.head()

,Aggression,Television,Computer_Games,Sibling_Aggression,Diet,Parenting_Style
0,0.374160,0.172671,0.141907,-0.328216,-0.110303,-0.279034
1,0.771153,-0.032872,0.709918,0.576837,-0.022990,-1.248167
2,-0.097728,-0.074460,-0.390141,-0.217184,0.280301,-0.328063
3,0.015935,-0.004427,-0.408080,0.046223,-0.263479,-1.005119
4,-0.275385,-0.675239,-0.277778,-0.891045,0.226581,0.489478


In [22]:
ols('Aggression ~ Television', child).fit().summary()

### skew: 보면 거의 대칭이 되어있음 +면 positive skew/ negative skew면 음수
### kurtosis(첨도) : 4 정도 -> 정규분포와 비슷 
### 등분산성 : 1.9 정도 (적당)
### 조건수 : 3.23 정도 (적당) 

## 다중공선성 - 조건수 

In [13]:
child['TV2'] = child['Television'] # 똑같은 변수 임의로 생성

In [ ]:
ols('Aggression ~ Television + TV2', child).fit().summary()
# 똑같은 변수인데 독립변수로 같이 보기 

### 위에만 보면 정상인데 밑에 vwarning 읽으면 multicolinearilty(다중공선성)
### cond.no.(조건수) : 거의 이억을 넘어감 ㅎㅎㅎ 이상하다는 거!! 

In [20]:
import seaborn

# 정규화

In [28]:
from sklearn.linear_model import LinearRegression, ElasticNetCV 
# elasticnet이 있고 cv가 붙은 게 있는 데 cv가 자동으로 해줌 

In [25]:
x = child.iloc[:,1:6] #1번열부터 6번열까지를 x
y = child['Aggression'] #aggression 열을 y

# 선형모형

In [29]:
m1 = LinearRegression()

In [30]:
m1.fit(x,y)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

In [31]:
m1.coef_
# 계수 추정하면 5개가 나옴 (x열의 순서대로) -> 아무 조건 없이 mse를 최소한 조건으로 추정하게 만들어 진 계수

array([ 0.03291628,  0.14216117,  0.08168417, -0.10905412,  0.0566481 ])

# 엘라스틱넷 + 교차검증
### 계수 + mse 둘 다 최소화 해줌

'l1_ratio' = 교안에서 a에 해당되는 부분 (왜? 라쏘(l1), 릿지(l2) 라고 함) ratio = 비율

In [35]:
m2 = ElasticNetCV(l1_ratio=[.1,.5,1], cv=3) #3-fold 교차검증 (데이터셋을 랜덤하게 3개로 나눈 후 2개로 추정한 것을 3번한다)
# 라쏘 후보 : 10%, 50%, 100% 라는 뜻 ,, 이거 정해지면 자동으로 l2도 정해짐(10% 면 l2는 90%)
m2.fit(x,y)
m2.coef_

array([ 0.02572348,  0.12471677,  0.0687051 , -0.08714782,  0.05652955])

In [36]:
# 그래서 나온 결과 해석 : 계수가 다 작은 수가 나옴 (선형모형하고 비교해서)

In [38]:
m2.l1_ratio_ #unerbar를 붙이는 건 사이킷런의 관습 (코드를 돌리고 나온 결과)
#l1_ratio가 0.1(10%)일때 일반화가 잘된다. (과적합이 제일 적게 된다)

0.1

In [39]:
m2.alpha_ #람다에 해당하는 부분 : 계수에다가 0.008 곱할 때 일반화가 잘된다는 뜻 

0.008882602491363027

## 예측을 잘하고 싶으면 -> 계수를 엘라스틱넷+교차검증 해서 나온 계수로 하는 게 더 좋다. 

# 더미코딩

In [43]:
hsb = pandas.read_csv('hsb2.csv') #미국 교육 데이터 

In [44]:
hsb.head() #ses : socio-economic status 사회경제적 수준
# schty : 학교 타입 #prog : 어떤 프로그램 , 그 뒤 5개는 교과목 이름 (socst : 사회)

,id,female,race,ses,schtyp,prog,read,write,math,science,socst
0,70,0,4,1,1,1,57,52,41,47,57
1,121,1,4,2,1,3,68,59,53,63,61
2,86,0,4,3,1,1,44,33,54,58,31
3,141,0,4,3,1,3,63,44,47,53,56
4,172,0,4,2,1,2,47,52,57,53,61


In [45]:
# 남자가 기준 0 아니면 여성 1 
# race는 인종 기준으로 그냥 번호 부여 - 회기 분석에 투입할 때 더미코딩해서 넣어야함 

In [46]:
ols('read ~C(race)', hsb).fit().summary()
# 더미코딩 C() 넣어주면 됨 

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                   read   R-squared:                       0.084
Model:                            OLS   Adj. R-squared:                  0.070
Method:                 Least Squares   F-statistic:                     5.964
Date:                Tue, 17 Sep 2019   Prob (F-statistic):           0.000654
Time:                        14:41:43   Log-Likelihood:                -740.06
No. Observations:                 200   AIC:                             1488.
Df Residuals:                     196   BIC:                             1501.
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
================================================================================
                   coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------
Intercept       46.6667      2.019     23.117      0.000      42.685      50.648
C(race)[T.2]     5.2424      3.601      1.456      0.147      -1.859      12.344
C(race)[T.3]     0.1333      2.994      0.045      0.965      -5.772       6.038
C(race)[T.4]     7.2575      2.179      3.330      0.001       2.959      11.556
==============================================================================
Omnibus:                        3.544   Durbin-Watson:                   1.966
Prob(Omnibus):                  0.170   Jarque-Bera (JB):                2.702
Skew:                           0.147   Prob(JB):                        0.259
Kurtosis:                       2.512   Cond. No.                         8.25
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

### 인종이 읽기 능력의 8.4% 영향을 미친다 
### 더미 코딩이 되서 계수가 3개가 됨 (abc순으로 정렬해서 기존을 그냥 맨 앞에 있는 걸 잡음 - 따로 정할 수도 있음 
- 1) 데이터 값을 바꾸는 방법 : 컬럼 지정해서 바꾸기
- 2) 더미코딩을 수동으로 해주는 방법)
### 계수를 보면 46.6 + 6.24*인종2 + 0.13*인종3 + 7.25*인종4 (계수 합) 
- -> 1번 기준이면 다른 계수 값은 다 0 이니까 그냥 46.4
- -> 인종 2 : 46.6 + 5.24 --> 그치만 신뢰구간이 -+ 니까 차이가 난다고 할 수 없다
- -> 인종 3 : 46.6 + 0.13 --> 그치만 신뢰구간이 -+ 니까 차이가 난다고 할 수 없다
- -> 인종 4 : 46.6 +7.25 --> 신뢰구간이 ++ 니까 차이가 난다 (4번인종이 1번인종보다 읽기 능력이 좋다 - 계수가 + 니까)


### 주의점! 전부 인종 1번하고만 비교하기 때문에 2번과 4번은 모름 (범위가 제한적)

In [52]:
### 2) 더미코딩을 수동으로 해주는 방법 
race = pandas.get_dummies(hsb['race'],prefix='race')
# prefix : 변수명 앞에 race라고 붙여라

In [62]:
new = pandas.concat([hsb, race], axis=1) #has와 dummy를 이어붙이 (concat)

In [63]:
ols('read ~ race_1 + race_2 + race_3', new).fit().summary()
# 기준으로 잡고 싶은 부분을 빼고 해야함 (지금은 race_4로 기준을 잡음)
# 지금 race_ 뒤에붙튼 1,2,3,4 는 숫자가 아니라 칼럼에 들어있던 값이다! 만약 백인, 황인, 흑인 이렇게 있었다면 
# race_백인, race_황인, race_흑인 이랬을 것... 

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                   read   R-squared:                       0.084
Model:                            OLS   Adj. R-squared:                  0.070
Method:                 Least Squares   F-statistic:                     5.964
Date:                Tue, 17 Sep 2019   Prob (F-statistic):           0.000654
Time:                        14:56:50   Log-Likelihood:                -740.06
No. Observations:                 200   AIC:                             1488.
Df Residuals:                     196   BIC:                             1501.
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     53.9241      0.821     65.658      0.000      52.304      55.544
race_1        -7.2575      2.179     -3.330      0.001     -11.556      -2.959
race_2        -2.0150      3.093     -0.652      0.515      -8.115       4.085
race_3        -7.1241      2.359     -3.020      0.003     -11.776      -2.472
==============================================================================
Omnibus:                        3.544   Durbin-Watson:                   1.966
Prob(Omnibus):                  0.170   Jarque-Bera (JB):                2.702
Skew:                           0.147   Prob(JB):                        0.259
Kurtosis:                       2.512   Cond. No.                         4.56
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [64]:
ols('read ~ race_2 + race_3 + race_4', new).fit().summary()
# 기준으로 잡고 싶은 부분을 빼고 해야함 (지금은 race_1로 기준을 잡음)
# 위에 자동으로 했던 부분하고 결과값이 같아야 함! 

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                   read   R-squared:                       0.084
Model:                            OLS   Adj. R-squared:                  0.070
Method:                 Least Squares   F-statistic:                     5.964
Date:                Tue, 17 Sep 2019   Prob (F-statistic):           0.000654
Time:                        14:56:52   Log-Likelihood:                -740.06
No. Observations:                 200   AIC:                             1488.
Df Residuals:                     196   BIC:                             1501.
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     46.6667      2.019     23.117      0.000      42.685      50.648
race_2         5.2424      3.601      1.456      0.147      -1.859      12.344
race_3         0.1333      2.994      0.045      0.965      -5.772       6.038
race_4         7.2575      2.179      3.330      0.001       2.959      11.556
==============================================================================
Omnibus:                        3.544   Durbin-Watson:                   1.966
Prob(Omnibus):                  0.170   Jarque-Bera (JB):                2.702
Skew:                           0.147   Prob(JB):                        0.259
Kurtosis:                       2.512   Cond. No.                         8.25
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [65]:
# ols('read ~ race', hsb).fit().summary() 
# 이 식이 안되는 이유! 돌리면 나오긴 하는 데 해석이 이상함...
# 인종이 2 증가할 때마다... 읽기 능력이 1 증가한다는 게 뭔 말이야? 라는 뜻이기 때문에! 

## 1) 더미코딩 데이터 값 바꾸는 방법 (칼럼 재설정)
## .replace로 컬럼값 바꾸기

In [71]:
hsb['race2'] = hsb['race'].replace({1:'흑', 2:'백', 3:'황', 4:'기타'})
# 임의로 컬럼값을 바꾼 후 해보면 가나다 순이니까 '기타' 가 기준이 된다

In [73]:
ols('read ~ C(race2)', hsb).fit().summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                   read   R-squared:                       0.084
Model:                            OLS   Adj. R-squared:                  0.070
Method:                 Least Squares   F-statistic:                     5.964
Date:                Tue, 17 Sep 2019   Prob (F-statistic):           0.000654
Time:                        15:08:40   Log-Likelihood:                -740.06
No. Observations:                 200   AIC:                             1488.
Df Residuals:                     196   BIC:                             1501.
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
=================================================================================
                    coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------
Intercept        53.9241      0.821     65.658      0.000      52.304      55.544
C(race2)[T.백]    -2.0150      3.093     -0.652      0.515      -8.115       4.085
C(race2)[T.황]    -7.1241      2.359     -3.020      0.003     -11.776      -2.472
C(race2)[T.흑]    -7.2575      2.179     -3.330      0.001     -11.556      -2.959
==============================================================================
Omnibus:                        3.544   Durbin-Watson:                   1.966
Prob(Omnibus):                  0.170   Jarque-Bera (JB):                2.702
Skew:                           0.147   Prob(JB):                        0.259
Kurtosis:                       2.512   Cond. No.                         4.56
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [ ]:
# 신뢰구간이 -- 면 비교했을 때 기타 계수보다 황과 흑의 읽기 능력이 더 낮다
# 그렇지만 백은 신뢰구간이 +- 이기 때문에 기타 계수랑 차이가 없다 

In [75]:
hsb['race3'] = hsb['race'].replace({1:'_흑', 2:'백', 3:'황', 4:'기타'})
# 임의로 컬럼값을 바꾼 후 해보면 가나다 순인데 특수문자가 있으니까 '_흑' 가 기준이 된다

In [76]:
ols('read ~ C(race3)', hsb).fit().summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                   read   R-squared:                       0.084
Model:                            OLS   Adj. R-squared:                  0.070
Method:                 Least Squares   F-statistic:                     5.964
Date:                Tue, 17 Sep 2019   Prob (F-statistic):           0.000654
Time:                        15:11:03   Log-Likelihood:                -740.06
No. Observations:                 200   AIC:                             1488.
Df Residuals:                     196   BIC:                             1501.
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
==================================================================================
                     coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------
Intercept         46.6667      2.019     23.117      0.000      42.685      50.648
C(race3)[T.기타]     7.2575      2.179      3.330      0.001       2.959      11.556
C(race3)[T.백]      5.2424      3.601      1.456      0.147      -1.859      12.344
C(race3)[T.황]      0.1333      2.994      0.045      0.965      -5.772       6.038
==============================================================================
Omnibus:                        3.544   Durbin-Watson:                   1.966
Prob(Omnibus):                  0.170   Jarque-Bera (JB):                2.702
Skew:                           0.147   Prob(JB):                        0.259
Kurtosis:                       2.512   Cond. No.                         8.25
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [77]:
# 계수 해석 
# 기타 46.6 + 7.25 -> 신뢰구간이 ++ 여서 기타가 흑보다 읽기 능력이 있단 뜻 
# 백 46.6 + 5.24(더 높은 계수값이 나왔지만 다른 샘플에선 달라짐) -> 신뢰구간이 -+ 딱히 차이 없음 
# 황 46.6 + 0.13(더 낮은 계수값이 나왔지만 다른 샘플에선 달라짐)  -> 신뢰구간이 -+ 딱히 차이 없음 

# 상호작용

In [79]:
hsb.head() #사회경제적 수준이 읽기에 영향을 줄까? 

,id,female,race,ses,schtyp,prog,read,write,math,science,socst,race2,race3
0,70,0,4,1,1,1,57,52,41,47,57,기타,기타
1,121,1,4,2,1,3,68,59,53,63,61,기타,기타
2,86,0,4,3,1,1,44,33,54,58,31,기타,기타
3,141,0,4,3,1,3,63,44,47,53,56,기타,기타
4,172,0,4,2,1,2,47,52,57,53,61,기타,기타


In [81]:
ols('read ~ ses', hsb).fit().summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                   read   R-squared:                       0.086
Model:                            OLS   Adj. R-squared:                  0.081
Method:                 Least Squares   F-statistic:                     18.64
Date:                Tue, 17 Sep 2019   Prob (F-statistic):           2.49e-05
Time:                        15:46:40   Log-Likelihood:                -739.80
No. Observations:                 200   AIC:                             1484.
Df Residuals:                     198   BIC:                             1490.
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     43.6972      2.095     20.858      0.000      39.566      47.829
ses            4.1522      0.962      4.317      0.000       2.256       6.049
==============================================================================
Omnibus:                        6.283   Durbin-Watson:                   1.816
Prob(Omnibus):                  0.043   Jarque-Bera (JB):                3.593
Skew:                           0.104   Prob(JB):                        0.166
Kurtosis:                       2.377   Cond. No.                         7.82
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [82]:
# 사회경제적 수준이 올라갈 수록 읽기 능력에 영향을 준다. 

In [83]:
hsb['schtyp2'] = hsb['schtyp'] - 1
# schtpy이 1,2 인데 0,1 데이터가 더 쉬우니까 1 빼서 만들어줌 

In [85]:
ols('read ~ ses + schtyp2', hsb).fit().summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                   read   R-squared:                       0.088
Model:                            OLS   Adj. R-squared:                  0.079
Method:                 Least Squares   F-statistic:                     9.529
Date:                Tue, 17 Sep 2019   Prob (F-statistic):           0.000112
Time:                        15:48:29   Log-Likelihood:                -739.57
No. Observations:                 200   AIC:                             1485.
Df Residuals:                     197   BIC:                             1495.
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     43.6743      2.098     20.816      0.000      39.537      47.812
ses            4.0613      0.972      4.178      0.000       2.144       5.979
schtyp2        1.3109      1.916      0.684      0.495      -2.467       5.089
==============================================================================
Omnibus:                        6.257   Durbin-Watson:                   1.804
Prob(Omnibus):                  0.044   Jarque-Bera (JB):                3.633
Skew:                           0.115   Prob(JB):                        0.163
Kurtosis:                       2.381   Cond. No.                         7.85
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [86]:
# 사회경제적 수준은 여전히 읽기 능력에 영향을 미치지만 스쿨타입은 신뢰구간이 -+ 니까 영향이 없다
# 사립이냐 공립이냐는 그닥 영향이 없다는 뜻 
# 그럼 상호작용을 어떻게 넣을까? 

In [88]:
ols('read ~ ses + ses:schtyp2', hsb).fit().summary()
# ses:schtyp2 -> 상호작용 (실제로는 곱하기 한 부분)

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                   read   R-squared:                       0.087
Model:                            OLS   Adj. R-squared:                  0.078
Method:                 Least Squares   F-statistic:                     9.376
Date:                Tue, 17 Sep 2019   Prob (F-statistic):           0.000129
Time:                        15:50:06   Log-Likelihood:                -739.71
No. Observations:                 200   AIC:                             1485.
Df Residuals:                     197   BIC:                             1495.
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
===============================================================================
                  coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------
Intercept      43.7565      2.104     20.799      0.000      39.608      47.905
ses             4.0600      0.987      4.114      0.000       2.114       6.006
ses:schtyp2     0.3568      0.822      0.434      0.665      -1.265       1.979
==============================================================================
Omnibus:                        6.259   Durbin-Watson:                   1.810
Prob(Omnibus):                  0.044   Jarque-Bera (JB):                3.603
Skew:                           0.108   Prob(JB):                        0.165
Kurtosis:                       2.379   Cond. No.                         7.99
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [89]:
# read = (4.06 + 0.35*학교) * ses -> 학교 타입 0(공립): 4.06*ses 고, 학교 타입 1(사립) 이면 4.41*ses 라는 뜻
# -> 학교 타입 0이 기울기가 더 완만하고, 학교 타입1이 기울기가 더 있다 (사립학교가 ses->성적에 미치는 영향이 더 크다) 라는 해석 
# 그러나!
# 사회경제적 수준과 상호작용한 스쿨타입은 신뢰구간이 -+ 니까 결국에는 영향이 없다
# = 학교에 따라서 ses -> 성적에 미치는 영향(돈있는 집이 더 공부를 잘한다)은 조절할 수 없다. 

In [90]:
# ses -> read 영향을 미치는 데, 
# 학교가 -> 사이에 들어가서 영향을 미칠까를 본거! -> 근데 영향이 없다는 결과가 나옴 

In [91]:
# 또다른 예시 
        #권한
        # |
        # ∨
# 업무량 ------- > 직무탈진  상호작용항 : 권한  (영향이 있다는 결과가 나왔음)
# (노트필기 참고)

### 그러면 ols('read ~ ses + chtyp2', hsb).fit().summary() -> 이거는 ses와 shtyp2가 read에 미치는 상관관계를 보는 거고
### ols('read ~ ses + ses:schtyp2', hsb).fit().summary() -> 이거는 ses가 read에 상관관계를 보이는 데 그게 학교에 따라서 영향을 받냐

In [93]:
# 또다른 예시 : ses를 성별로 - write에 영향
ols('write ~ ses + female:ses', hsb).fit().summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  write   R-squared:                       0.116
Model:                            OLS   Adj. R-squared:                  0.107
Method:                 Least Squares   F-statistic:                     12.90
Date:                Tue, 17 Sep 2019   Prob (F-statistic):           5.42e-06
Time:                        16:08:54   Log-Likelihood:                -720.78
No. Observations:                 200   AIC:                             1448.
Df Residuals:                     197   BIC:                             1457.
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     46.6663      1.914     24.378      0.000      42.891      50.441
ses            1.7444      0.909      1.918      0.057      -0.049       3.538
female:ses     2.3479      0.583      4.027      0.000       1.198       3.498
==============================================================================
Omnibus:                       11.857   Durbin-Watson:                   1.792
Prob(Omnibus):                  0.003   Jarque-Bera (JB):                7.477
Skew:                          -0.324   Prob(JB):                       0.0238
Kurtosis:                       2.309   Cond. No.                         8.80
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [94]:
# write = ses + ses:female
# write = (1.7+2.3female)ses
# -> 남자는 0 여자는 1 : write = (1.7)ses -> 남자는 1올라갈 때 1.7 쓰기 능력이 오름
# -> 여자 1 : write = (4.0)ses -> 여자는 1올라갈 때 4.0 쓰기 능력이 오름 

In [97]:
ols('write ~ ses + female:ses + female', hsb).fit().summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  write   R-squared:                       0.124
Model:                            OLS   Adj. R-squared:                  0.111
Method:                 Least Squares   F-statistic:                     9.256
Date:                Tue, 17 Sep 2019   Prob (F-statistic):           9.38e-06
Time:                        16:17:21   Log-Likelihood:                -719.84
No. Observations:                 200   AIC:                             1448.
Df Residuals:                     196   BIC:                             1461.
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     43.3109      3.120     13.881      0.000      37.157      49.464
ses            3.1618      1.382      2.288      0.023       0.437       5.887
female:ses     0.0388      1.795      0.022      0.983      -3.501       3.578
female         5.3668      3.946      1.360      0.175      -2.415      13.149
==============================================================================
Omnibus:                       11.529   Durbin-Watson:                   1.789
Prob(Omnibus):                  0.003   Jarque-Bera (JB):                7.780
Skew:                          -0.349   Prob(JB):                       0.0205
Kurtosis:                       2.332   Cond. No.                         22.5
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [98]:
# 상호관계뿐만 아니라 상관관계까지 넣어서 본다면? 
# write = ses + ses:female + female
# write = 3.16ses +0.03ses:female + 5.36female
# -> 남자는 0 여자는 1 : -> 남자는 1올라갈 때 3.16 쓰기 능력이 오름
# -> 여자 1 :  -> 여자는 1올라갈 때 3.19 쓰기 능력이 오름 
# 그러나!
# 신뢰구간이 +- 기 때문에 의미가 없다. 

In [100]:
# 또다른 예 : 읽기를 잘하면 쓰기도 잘할까, 성별하고 상호작용이 있을 까
ols('write ~ read+female:read', hsb).fit().summary() #female:read 상호작용

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  write   R-squared:                       0.429
Model:                            OLS   Adj. R-squared:                  0.423
Method:                 Least Squares   F-statistic:                     73.90
Date:                Tue, 17 Sep 2019   Prob (F-statistic):           1.13e-24
Time:                        16:24:30   Log-Likelihood:                -677.12
No. Observations:                 200   AIC:                             1360.
Df Residuals:                     197   BIC:                             1370.
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
===============================================================================
                  coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------
Intercept      23.2004      2.654      8.741      0.000      17.966      28.434
read            0.5144      0.050     10.218      0.000       0.415       0.614
female:read     0.0961      0.019      5.000      0.000       0.058       0.134
==============================================================================
Omnibus:                        7.827   Durbin-Watson:                   1.974
Prob(Omnibus):                  0.020   Jarque-Bera (JB):                4.749
Skew:                          -0.201   Prob(JB):                       0.0931
Kurtosis:                       2.361   Cond. No.                         322.
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

### read -> write에 영향을 미치는 데 
### read에서 성별이 영향을 미치고 결국 write까지 영향을 미침 
- -> 남자 0 : w = 0.5r
- -> 여자 1 : w = 0.6r

In [103]:
# 상호작용이 있는 경우 (2)
ols('write ~ female*read', hsb).fit().summary() 
# 'write ~ read + female + read:female' 이 뜻과 같다. (female + female:read와 상호작용 하라는 뜻 *:은 곱하기라서 순서 상관없음!!* ) 

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  write   R-squared:                       0.445
Model:                            OLS   Adj. R-squared:                  0.436
Method:                 Least Squares   F-statistic:                     52.31
Date:                Tue, 17 Sep 2019   Prob (F-statistic):           7.00e-25
Time:                        16:34:54   Log-Likelihood:                -674.28
No. Observations:                 200   AIC:                             1357.
Df Residuals:                     196   BIC:                             1370.
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
===============================================================================
                  coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------
Intercept      16.5239      3.845      4.297      0.000       8.941      24.107
female         12.4906      5.259      2.375      0.019       2.119      22.863
read            0.6360      0.071      8.907      0.000       0.495       0.777
female:read    -0.1339      0.099     -1.357      0.176      -0.328       0.061
==============================================================================
Omnibus:                        5.511   Durbin-Watson:                   1.972
Prob(Omnibus):                  0.064   Jarque-Bera (JB):                4.075
Skew:                          -0.222   Prob(JB):                        0.130
Kurtosis:                       2.459   Cond. No.                         751.
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""